In [28]:
import numpy as np
import pandas as pd
import time, gc
pi = np.pi 
cos = np.cos 
sin = np.sin
acos = np.arccos
degrees = np.degrees
radians = np.radians

In [104]:
def angularDistance(row):
    '''
    Computes the angular distance (degrees) between the reference Right-Ascension (RA)
    & Declination (Dec) value, and the corresponding RA - Dec value of the stars in 
    the dataframe - <OB_CATALOGUE>
    
    <Formula> - http://spiff.rit.edu/classes/phys373/lectures/radec/radec.html
    
    Parameters
    ----------
    row : pd.Dataframe - series
        Input RA/Dec in degrees from the <IMG_DF> dataframe
          
    Returns
    -------
    y : pd.Dataframe - series
        The corresponding angular distance in degree value.
    '''
    # Units of right-ascension is in (hours) format
    # Units of declination is in (degrees) format
    alpha1, alpha2 = radians(15*row['RA_1']), radians(15*row['RA_2'])
    delta1, delta2 = radians(row['Dec_1']), radians(row['Dec_2'])
    temp = cos(pi/2 - delta1)*cos(pi/2 - delta2) + sin(pi/2 - delta1)*sin(pi/2 - delta2)*cos(alpha1 - alpha2) 
    return np.degrees(acos(temp))

In [ ]:
CATALOGUE = pd.read_csv("Modified Star Catalogue.csv")
#StarID: The database primary key from a larger "master database" of stars
#Mag: The star's apparent visual magnitude
#RA, Dec: The star's right ascension and declination, for epoch 2000.0 (Unit: RA - hrs; Dec - degrees)
CATALOGUE.sort_values('Mag', inplace=True)

In [81]:
CATALOGUE.head()

,StarID,RA,Dec,Mag
0,23440,6.752569,-16.713143,-1.44
1,21936,6.399192,-52.695718,-0.62
2,50628,14.261208,19.187270,-0.05
3,51926,14.661361,-60.835147,-0.01
4,65795,18.615607,38.782993,0.03


In [262]:
def genRefCatalogue(mag_limit, no_iter = -1, gen_csv = True):
    '''
    Generates the reference star catalogue for Geometric Voting Algorithm where each row
    of the table has two unique stars and the corresponding angular distance in degrees,
    for all pairs of stars with a specified upper magnitude limit
    
    Parameters
    ----------
    mag_limit : floating-point number
        The upper magnitude limit of stars that are required in the reference catalogue
        
    no_iter : integer, default = -1
        Specifies the number of iterations, thereby allowing it to be reduced
        Default value = -1, allows for the completion of the entire catalogue
        
    gen_csv : boolean, default = True
          If True generates csv files of the reference catalogues
          
    Returns
    -------
    OB_CATALOGUE : pd.Dataframe
        The corresponding angular distance in degree value.
    '''
    
    # Start clock-1
    start1 = time.time()
    
    # Generate restricted catalogue based on upper magnitude limit
    temp0 = CATALOGUE[CATALOGUE.Mag <= mag_limit]
    
    # Number of rows in the resticted catalogue
    rows = temp0.shape[0]
    # Resets the index of <temp0>
    temp0.index = list(range(rows))
    
    # Prints total number of stars in <temp0> and the (n)C(2) - combinations
    print('Number of stars - ', rows)
    print('Number of unique combinations = ', (rows-1)*rows/2)
    
    # Initialize the number of iterations to take place
    no_iter = (rows-1) if no_iter == -1 else no_iter
    
    for i in range(no_iter):
        # Throws error if an iteration runs beyond number of available rows in <temp0>
        assert i<(rows-1), 'iterating beyond available number of rows'
        
        # The final iteration is reduntant, as <temp2> will be zero rows
        '''
        if (rows-1-i)==0:
            continue
        '''
        
        # Generates <temp1> dataframe which has the (i - th) star of <temp0>
        # repetated (rows-1-i) times 
        temp1 = pd.DataFrame(columns = ['Star_ID1','RA_1', 'Dec_1', 'Mag_1'])
        s1, ra, dec, mag = temp0.iloc[i]
        temp1.loc[0] = [s1] + [ra] + [dec] + [mag]
        temp1 = pd.concat([temp1]*(rows-1-i), ignore_index=True)
        
        # Generates <temp2> dataframe by copying values of <temp0> and dropping the first
        # (i + 1) number of stars
        temp2 = temp0
        temp2 = temp2.drop(list(range(i+1)), axis = 0)
        # Resets the index 
        temp2.index = list(range(0, rows-1-i))
        
        # Concatenates <temp1> & <temp2> side-by-side such that resulting <temp3> has (8) columns altogether
        temp3 = pd.concat([temp1, temp2], axis=1)
        
        # Initializes <temp4> in the first iteratation
        if i == 0:
            temp4 = temp3
            
        # Append subsequent <temp4> with <temp3> after first iteration
        else:
            temp4 = pd.concat([temp4, temp3], axis = 0, ignore_index=True)
        
        # Releases memory back to OS
        if i%40 == 0:
            gc.collect()
    
    gc.collect()      
    # Rename columns
    temp4.columns = ['Star_ID1','RA_1', 'Dec_1', 'Mag_1', 'Star_ID2', 'RA_2', 'Dec_2', 'Mag_2']
    
    if gen_csv == True:
        #Generates CSV of <temp4>
        temp4.to_csv('Processed_Catalogue1.csv', index = False)
        
    # Stop clock-1   
    end1 = time.time() - start1
    
    # Print time taken
    print('Process 1 - ', end1)
    
    # Start clock-2
    start2 = time.time()
    
    # Initialize <OB_CATALOGUE>
    OB_CATALOGUE = temp4
    
    # Calculate angular distance between the two stars present in every row
    OB_CATALOGUE['Ang_Distance'] = OB_CATALOGUE.apply(angularDistance, axis = 1)
    
    if gen_csv == True:
        # Generates CSV of <OB_CATALOGUE>
        OB_CATALOGUE.to_csv('Processed_Catalogue2.csv', index = False)
        
    # Stop clock-2
    end2 = time.time() - start2
    
    # Print time taken
    print('Process 2 - ', end2)
    print('Total Process ', end1+ end2)
        
    return OB_CATALOGUE

In [264]:
t = genRefCatalogue(mag_limit=4, no_iter=-1, gen_csv=False)
t.tail()

Number of stars -  523
Number of unique combinations =  136503.0
Process 1 -  20.131972074508667
Process 2 -  7.736001968383789
Total Process  27.867974042892456


,Star_ID1,RA_1,Dec_1,Mag_1,Star_ID2,RA_2,Dec_2,Mag_2,Ang_Distance
136498,6503.0,2.000062,-21.077772,3.99,41155,11.398714,10.529698,4.0,141.119413
136499,6503.0,2.000062,-21.077772,3.99,57143,16.199928,-19.460647,4.0,128.174007
136500,32817.0,9.040777,-66.395844,4.00,41155,11.398714,10.529698,4.0,81.166643
136501,32817.0,9.040777,-66.395844,4.00,57143,16.199928,-19.460647,4.0,78.903303
136502,41155.0,11.398714,10.529698,4.00,57143,16.199928,-19.460647,4.0,76.979891


In [253]:
t.sort_values('Ang_Distance', inplace=False)

,Star_ID1,RA_1,Dec_1,Mag_1,Star_ID2,RA_2,Dec_2,Mag_2,Ang_Distance
70138,45451.0,12.694445,-1.449522,2.74,86675,12.694690,-1.449330,3.52,0.003673
51588,60441.0,17.172962,-15.725148,2.43,86977,17.172749,-15.728212,3.40,0.004341
42389,47710.0,13.398728,54.925415,2.23,86727,13.398899,54.920892,3.95,0.004757
15059,26923.0,7.576668,31.888636,1.58,86307,7.576978,31.891883,2.85,0.005110
1916,51926.0,14.661361,-60.835147,-0.01,51923,14.660942,-60.839471,1.35,0.005298
3168,17425.0,5.278138,45.999029,0.08,86179,5.277851,46.006709,0.96,0.008238
130592,14714.0,4.477687,15.870947,3.40,14706,4.476231,15.962217,3.84,0.093657
96320,59345.0,16.864511,-38.047327,3.00,59369,16.872265,-38.017478,3.56,0.096354
42410,47710.0,13.398728,54.925415,2.23,47776,13.420393,54.987999,3.99,0.196818
191585,86727.0,13.398899,54.920892,3.95,47776,13.420393,54.987999,3.99,0.196929


# Testing

In [254]:
temp0 = CATALOGUE[CATALOGUE.Mag <= 0]
rows = temp0.shape[0]
temp0.index = list(range(rows))
#print(rows)
#junk_0 = []
#junk_01 = []
for i in range(rows):
    temp1 = pd.DataFrame(columns = ['Star_ID1','RA_1', 'Dec_1', 'Mag_1'])
    s1, ra, dec, mag = temp0.iloc[i]
    #print(s1, i)
    if (rows-1-i)==0:
        continue
        
    temp1.loc[0] = [s1] + [ra] + [dec] + [mag]
        
    temp1 = pd.concat([temp1]*(rows-1-i), ignore_index=True)

    temp2 = temp0
    temp2 = temp2.drop(list(range(i+1)), axis = 0)
    temp2.index = list(range(0, rows-1-i))
    ###################
    #junk_0.append(temp1.shape[0])
    #junk_01.append(temp2.shape[0])
    ##################
    if(i==8):
        junk1 = temp1
        junk2 = temp2

    temp3 = pd.concat([temp1, temp2], axis=1)
    temp3.head()

    if i == 0:
        temp4 = temp3
    else:
        temp4 = pd.concat([temp4, temp3], axis = 0, ignore_index=True)
        
    if i%20 == 0:
            gc.collect()
            
temp4.columns = ['Star_ID1','RA_1', 'Dec_1', 'Mag_1', 'Star_ID2', 'RA_2', 'Dec_2', 'Mag_2']
#temp4.to_csv('Processed_Catalogue1.csv')
OB_CATALOGUE = temp4
OB_CATALOGUE['Ang_Distance'] = OB_CATALOGUE.apply(angularDistance, axis = 1)
#OB_CATALOGUE.to_csv('Processed_Catalogue2.csv')
#OB_CATALOGUE.tail()

In [255]:
temp0

,StarID,RA,Dec,Mag
0,23440,6.752569,-16.713143,-1.44
1,21936,6.399192,-52.695718,-0.62
2,50628,14.261208,19.187270,-0.05
3,51926,14.661361,-60.835147,-0.01


In [256]:
OB_CATALOGUE

,Star_ID1,RA_1,Dec_1,Mag_1,Star_ID2,RA_2,Dec_2,Mag_2,Ang_Distance
0,23440.0,6.752569,-16.713143,-1.44,21936,6.399192,-52.695718,-0.62,36.223936
1,23440.0,6.752569,-16.713143,-1.44,50628,14.261208,19.187270,-0.05,116.267431
2,23440.0,6.752569,-16.713143,-1.44,51926,14.661361,-60.835147,-0.01,88.425907
3,21936.0,6.399192,-52.695718,-0.62,50628,14.261208,19.187270,-0.05,121.973312
4,21936.0,6.399192,-52.695718,-0.62,51926,14.661361,-60.835147,-0.01,58.013449
5,50628.0,14.261208,19.187270,-0.05,51926,14.661361,-60.835147,-0.01,80.169175


In [257]:
OB_CATALOGUE.sort_values('Ang_Distance', inplace=False)

,Star_ID1,RA_1,Dec_1,Mag_1,Star_ID2,RA_2,Dec_2,Mag_2,Ang_Distance
0,23440.0,6.752569,-16.713143,-1.44,21936,6.399192,-52.695718,-0.62,36.223936
4,21936.0,6.399192,-52.695718,-0.62,51926,14.661361,-60.835147,-0.01,58.013449
5,50628.0,14.261208,19.187270,-0.05,51926,14.661361,-60.835147,-0.01,80.169175
2,23440.0,6.752569,-16.713143,-1.44,51926,14.661361,-60.835147,-0.01,88.425907
1,23440.0,6.752569,-16.713143,-1.44,50628,14.261208,19.187270,-0.05,116.267431
3,21936.0,6.399192,-52.695718,-0.62,50628,14.261208,19.187270,-0.05,121.973312
